<img src="./img/banner.png">

# Laboratorio 2 - BancAlpes

1. Carga y exploración de datos.

    1.1. Importación de librerias.
    
    1.2. Carga de datos.
    
    1.3. Exploración.
    
2. Limpieza y preparación de los datos.

    2.1. Tipo adecuado de cada atributo.
    
    2.2. Selección de atributos.
    
3. Clasificadores.

    3.1.1. K-means.
    
    3.1.1. Clustering jerárquico.
    
    3.1.1.

## 1. Carga y exploración de datos.

### 1.1. Importación de librerias.

In [191]:
# Librerías para manejo de datos
import pandas as pd
import numpy as np

# preparacion de los datos

#Librerías para la visualización
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline


### 1.2. Carga de datos.

In [192]:
# carga de los datos
# csv -> dataFrame
df_clients = pd.read_csv('./datos/BancAlpes_credit_card_data.csv', sep=';', encoding = 'utf-8' )

### 1.3. Exploración.

In [193]:
#cantidad de datos y numero de variables
df_clients.shape

(660, 11)

In [194]:
# mostrar 5 filas al azar
df_clients.sample(5)

,Id,Customer,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
216,217,54477,240000,Female,2,Married,36,3,1,5,4
360,361,15318,500000,Female,2,Single,23,6,3,0,1
432,433,37252,110000,Female,1,Single,29,6,2,1,2
573,574,96534,160000,Female,2,Single,60,7,2,2,3
609,610,11562,200000,Female,2,Married,34,4,3,2,0


In [195]:
# ver todos los tipos de variables 
df_clients.dtypes

Id                      int64
Customer               object
Limit_bal              object
Sex                    object
Education              object
Marriage               object
Age                    object
Total_Credit_Cards     object
Total_visits_bank      object
Total_visits_online    object
Total_calls_made       object
dtype: object

In [196]:
#cantidad de datos nulos
df_clients.isnull().sum()

Id                     0
Customer               0
Limit_bal              0
Sex                    0
Education              0
Marriage               0
Age                    0
Total_Credit_Cards     0
Total_visits_bank      0
Total_visits_online    0
Total_calls_made       0
dtype: int64

No existen datos nulos.

## 2. Limpieza y preparación de los datos.

### 2.1. Tipo adecuado de cada atributo.

Primero realizaremos el cambio de tipo de cada columna, acorde con el tipo de datos que maneja.

In [197]:
# ver todos los tipos de variables 
df_clients.dtypes

Id                      int64
Customer               object
Limit_bal              object
Sex                    object
Education              object
Marriage               object
Age                    object
Total_Credit_Cards     object
Total_visits_bank      object
Total_visits_online    object
Total_calls_made       object
dtype: object

Podemos observar que casi todos los datos los identifica como `object`, entonces procederemos a cambiarlos a su tipo adecuado.

Al intentar cambiar el tipo de cada columna a entero, nos arroja un error: ` ValueError: invalid literal for int() with base 10: '-' `
Esto es debido a que existen datos o caracteres que no corresponden en la columna a cambiar, en este caso existe el caracter '-' el cual no nos permite cambiar el tipo de la columna

In [198]:
# buscamos la fila con el caracter anomalo '-'
raro = df_clients[df_clients['Customer'].str.contains('-')]
print(raro)

    Id Customer Limit_bal     Sex Education Marriage Age Total_Credit_Cards  \
11  12        -    260000  Female         1        -  51                  3   
31  32        -         -       -         -        -   -                  -   

   Total_visits_bank Total_visits_online Total_calls_made  
11                 -                   2                7  
31                 -                   -                -  


Como podemos observar las filas 11 y 32  contienen varias columnas con este valor, lo cual no aporta informacion relevante para los modelos de ML y seran eliminadas

In [199]:
# eliminamos la fila que contiene '-' en la columna Customer
df_clients = df_clients.drop(df_clients[df_clients.Customer.str.contains('-')].index)

Ahora convertimos la columna a su correspondiente tipo

In [200]:
df_clients['Customer'] = df_clients.Customer.astype(int)
df_clients['Limit_bal'] = df_clients.Limit_bal.astype(int)

 Al intentar cambiar el tipo de la columna **Education** entero, nos arroja un error: ´ValueError: invalid literal for int() with base 10: 'ABC´´

In [201]:
#verificos el tipo de datos manejados por la columna
df_clients.Education.value_counts()

2      288
1      263
3       99
5        3
4        2
6        2
ABC      1
Name: Education, dtype: int64

podemos ver que solo hay una fila con los caracteres 'ABC'

In [202]:
# buscamos la fila con el caracter anomalo 'ABC'
raro2 = df_clients[df_clients['Education'].str.contains('ABC')]
print(raro2)

    Id  Customer  Limit_bal    Sex Education Marriage Age Total_Credit_Cards  \
15  16     55196      50000  Fmale       ABC   Others  23                  2   

   Total_visits_bank Total_visits_online Total_calls_made  
15                 0                   5                7  


consideramos que al no tener precisión en el tipo de sexo (*Fmale*) y la educación (*ABC*), la fila sera eliminada de los datos.

In [203]:
# eliminamos la fila que contiene 'ABC' en la columna Education
df_clients = df_clients.drop(df_clients[df_clients.Education.str.contains('ABC')].index)

In [204]:
#Ahora convertimos la columna a su correspondiente tipo
df_clients['Education'] = df_clients.Education.astype(int)
df_clients['Age'] = df_clients.Age.astype(int)

In [205]:
# buscamos la fila con el caracter anomalo '-'
raro3 = df_clients[df_clients['Total_Credit_Cards'].str.contains('-')]
print(raro3)

    Id  Customer  Limit_bal   Sex  Education Marriage  Age Total_Credit_Cards  \
17  18     96463     320000  Male          1  Married   49                  -   

   Total_visits_bank Total_visits_online Total_calls_made  
17                 0                   3                4  


Consideramos valiosa la fila y remplazamos el valor de esta por 1, debido a que existe un monto de prestamo en el registro de *Limit_bal*.

In [206]:

df_clients.Total_Credit_Cards = df_clients.Total_Credit_Cards.replace({'-':1})

#comprobamos que no exista el caracter '-' en la columna
print( df_clients[df_clients['Total_Credit_Cards']=='-'] )

#convertimos la columna en entero
df_clients['Total_Credit_Cards'] = df_clients.Total_Credit_Cards.astype(int)



Empty DataFrame
Columns: [Id, Customer, Limit_bal, Sex, Education, Marriage, Age, Total_Credit_Cards, Total_visits_bank, Total_visits_online, Total_calls_made]
Index: []


In [207]:
# buscamos la fila con el caracter anomalo '?'
raro4 = df_clients[df_clients['Total_visits_bank'].str.contains('\?')]
print(raro4)

    Id  Customer  Limit_bal   Sex  Education Marriage  Age  \
16  17     62617      20000  Male          1   Single   24   
38  39     52142      50000  Male          1   Single   25   

    Total_Credit_Cards Total_visits_bank Total_visits_online Total_calls_made  
16                   2                 ?                   2                4  
38                   4                 ?                   3               10  


Consideramos valiosa las filas y remplazamos sus valores por 1, debido a que existe la posibilidad de visitar el banco por lo menos una vez para abrir una cuenta.

In [208]:
df_clients.Total_visits_bank = df_clients.Total_visits_bank.replace({'?':1})

#comprobamos el cambio
df_clients.loc[16]


Id                         17
Customer                62617
Limit_bal               20000
Sex                      Male
Education                   1
Marriage               Single
Age                        24
Total_Credit_Cards          2
Total_visits_bank           1
Total_visits_online         2
Total_calls_made            4
Name: 16, dtype: object

In [209]:
#comprobamos el cambio
df_clients.loc[38]

Id                         39
Customer                52142
Limit_bal               50000
Sex                      Male
Education                   1
Marriage               Single
Age                        25
Total_Credit_Cards          4
Total_visits_bank           1
Total_visits_online         3
Total_calls_made           10
Name: 38, dtype: object

In [210]:
df_clients['Total_visits_bank'] = df_clients.Total_visits_bank.astype(int)
df_clients['Total_visits_online'] = df_clients.Total_visits_online.astype(int)

In [211]:
# buscamos la fila con el caracter anomalo '?'
raro5 = df_clients[df_clients['Total_calls_made'].str.contains('\?')]
print(raro5)

   Id  Customer  Limit_bal     Sex  Education Marriage  Age  \
3   4     40496      50000  Female          2  Married   37   

   Total_Credit_Cards  Total_visits_bank  Total_visits_online Total_calls_made  
3                   5                  1                    1                ?  


Consideramos valiosa la fila y remplazamos su valor por 1, debido a que solo ha realizado una vista al banco y una online.

In [212]:
df_clients.Total_calls_made = df_clients.Total_calls_made.replace({'?':1})

#comprobamos el cambio
df_clients.loc[3]

Id                           4
Customer                 40496
Limit_bal                50000
Sex                     Female
Education                    2
Marriage               Married
Age                         37
Total_Credit_Cards           5
Total_visits_bank            1
Total_visits_online          1
Total_calls_made             1
Name: 3, dtype: object

In [213]:
#cambiamos el tipo de la columna
df_clients['Total_calls_made'] = df_clients.Total_calls_made.astype(int)

In [214]:
#comprobamos los tipos de las columnas
df_clients.dtypes

Id                      int64
Customer                int64
Limit_bal               int64
Sex                    object
Education               int64
Marriage               object
Age                     int64
Total_Credit_Cards      int64
Total_visits_bank       int64
Total_visits_online     int64
Total_calls_made        int64
dtype: object

Ahora convertimos las columnas de *Sex* y *Marriage* en datos numericos para poder integralos al modelo.

In [215]:
#comprobamos los datos de Sex
df_clients.Sex.value_counts()

Female    371
Male      273
M           3
F           3
Mael        3
f           2
Femael      1
9           1
Name: Sex, dtype: int64

In [216]:
# buscamos la fila con el caracter anomalo '9'
raro6 = df_clients[df_clients['Sex'].str.contains('9')]
print(raro6)

    Id  Customer  Limit_bal Sex  Education Marriage  Age  Total_Credit_Cards  \
70  71     21976      80000   9          1   Single   31                   1   

    Total_visits_bank  Total_visits_online  Total_calls_made  
70                  2                    4                 7  


consideramos borrar esta fila, debido a que creemos que impactara en el modelo.

In [217]:
df_clients = df_clients.drop([70], axis=0)

#comprobamos si se elimino.
raro6 = df_clients[df_clients['Sex'].str.contains('9')]
print(raro6)

Empty DataFrame
Columns: [Id, Customer, Limit_bal, Sex, Education, Marriage, Age, Total_Credit_Cards, Total_visits_bank, Total_visits_online, Total_calls_made]
Index: []


In [218]:
#reemplazamos los M => Male
df_clients.Sex = df_clients.Sex.replace({'M ':'Male'})

#reemplazamos los F => Female
df_clients.Sex = df_clients.Sex.replace({'F':'Female'})

#reemplazamos los Mael => Male
df_clients.Sex = df_clients.Sex.replace({'Mael':'Male'})

#reemplazamos los f => Female
df_clients.Sex = df_clients.Sex.replace({'f':'Female'})

#reemplazamos los Femael => Female
df_clients.Sex = df_clients.Sex.replace({'Femael':'Female'})

In [219]:
#comprobamos los datos de Sex
df_clients.Sex.value_counts()

Female    377
Male      279
Name: Sex, dtype: int64

In [220]:
#mapping para la columna de sex
sexMaping = {'Female':0,'Male':1}
df_clients['Sex'] = df_clients['Sex'].map(sexMaping)

#comprobamos el map
df_clients.Sex.value_counts()

0    377
1    279
Name: Sex, dtype: int64

In [221]:
df_clients.Marriage.value_counts()

Single     378
Married    260
Others       8
1            5
2            3
?            1
0            1
Name: Marriage, dtype: int64

In [222]:
#reemplazamos los 1 => Others
df_clients.Marriage = df_clients.Marriage.replace({ '1':'Others'})

#reemplazamos los 2 => Others
df_clients.Marriage = df_clients.Marriage.replace({ '2':'Others'})

#reemplazamos los ? => Others
df_clients.Marriage = df_clients.Marriage.replace({ '?':'Others'})

#reemplazamos los ? => Others
df_clients.Marriage = df_clients.Marriage.replace({ '0' :'Others'})

In [223]:
#comprobamos los cambios
df_clients.Marriage.value_counts()

Single     378
Married    260
Others      18
Name: Marriage, dtype: int64

In [224]:
#mapping para la columna de sex
MarriageMaping = {'Single':0,'Married':1,'Others':2}
df_clients['Marriage'] = df_clients['Marriage'].map(MarriageMaping)

#comprobamos el map
df_clients.Marriage.value_counts()

0    378
1    260
2     18
Name: Marriage, dtype: int64

In [225]:
#Finalmente las estadisiticas descriptivas.
df_clients.describe()

,Id,Customer,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
count,656.000000,656.000000,656.000000,656.000000,656.000000,656.000000,656.000000,6.560000e+02,656.000000,656.000000,656.000000
mean,332.315549,55230.647866,170182.926829,0.425305,1.783537,0.451220,95.493902,1.882010e+05,2.413110,2.597561,3.559451
std,189.813037,25657.650223,129855.927499,0.494766,0.778786,0.550418,1435.931970,4.820178e+06,1.630364,2.941090,2.864812
min,1.000000,11265.000000,10000.000000,0.000000,1.000000,0.000000,21.000000,1.000000e+00,0.000000,0.000000,0.000000
25%,168.750000,33952.500000,50000.000000,0.000000,1.000000,0.000000,28.000000,3.000000e+00,1.000000,1.000000,1.000000
50%,332.500000,53907.000000,140000.000000,0.000000,2.000000,0.000000,33.500000,5.000000e+00,2.000000,2.000000,3.000000
75%,496.250000,77439.000000,240000.000000,1.000000,2.000000,1.000000,41.000000,6.000000e+00,4.000000,4.000000,5.000000
max,660.000000,99843.000000,630000.000000,1.000000,6.000000,2.000000,36745.000000,1.234568e+08,5.000000,15.000000,10.000000


### 2.2. Selección de atributos.

En este punto deseamos conocer cuales son las columnas más importantes para nuestro modelo, aquellas que aporten valor y sirvan para crear modelos más precisos y exactos. Para este punto necesitamos conocimiento sobre cada atributo y como este podria impactar en la prediccion del modelo.

Eliminamos las columnas de **Id** y de **Customer** al no aportar nada para el rendimiento del modelo

In [226]:
df_clients_A = df_clients
del df_clients_A['Id']
del df_clients_A['Customer']

In [227]:
#comprobamos la eliminacion
df_clients_A.dtypes

Limit_bal              int64
Sex                    int64
Education              int64
Marriage               int64
Age                    int64
Total_Credit_Cards     int64
Total_visits_bank      int64
Total_visits_online    int64
Total_calls_made       int64
dtype: object

In [228]:
# verificamos los rangos
df_clients_A.describe()

,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
count,656.000000,656.000000,656.000000,656.000000,656.000000,6.560000e+02,656.000000,656.000000,656.000000
mean,170182.926829,0.425305,1.783537,0.451220,95.493902,1.882010e+05,2.413110,2.597561,3.559451
std,129855.927499,0.494766,0.778786,0.550418,1435.931970,4.820178e+06,1.630364,2.941090,2.864812
min,10000.000000,0.000000,1.000000,0.000000,21.000000,1.000000e+00,0.000000,0.000000,0.000000
25%,50000.000000,0.000000,1.000000,0.000000,28.000000,3.000000e+00,1.000000,1.000000,1.000000
50%,140000.000000,0.000000,2.000000,0.000000,33.500000,5.000000e+00,2.000000,2.000000,3.000000
75%,240000.000000,1.000000,2.000000,1.000000,41.000000,6.000000e+00,4.000000,4.000000,5.000000
max,630000.000000,1.000000,6.000000,2.000000,36745.000000,1.234568e+08,5.000000,15.000000,10.000000


Encontramos anomalias en las columnas de **Age** y de **Total_Credit_Cards**

In [229]:
df_clients_A[ df_clients_A['Age'] > 130 ]

,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
42,10000,1,2,0,2202,4,2,3,4
46,20000,0,1,0,36745,1,2,2,5
50,70000,1,3,2,789,1,0,5,9


Consideramos eliminar estas filas, debido a que desconocemos la edad real. Debido a que estas pueden impactar en el rendimiento del modelo.

In [230]:
df_clients_A = df_clients_A.drop([42,46,50], axis=0)

#comprobamos
df_clients_A[ df_clients_A['Age'] > 130 ]


,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made


In [233]:
df_clients_A[ df_clients_A['Total_Credit_Cards'] > 10 ]

,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
74,340000,1,1,0,32,123456789,0,5,8


In [234]:
#eliminamos la fila debido al numero anormal de tarjtas de credito
df_clients_A = df_clients_A.drop([74], axis=0)

#comprobamos
df_clients_A[ df_clients_A['Total_Credit_Cards'] > 10 ]

,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made


In [235]:
# verificamos los rangos
df_clients_A.describe()

,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
count,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000
mean,170552.147239,0.423313,1.783742,0.450920,35.085890,4.736196,2.421779,2.590491,3.541411
std,129739.199740,0.494463,0.778462,0.547904,9.359618,2.161180,1.629703,2.946961,2.859721
min,10000.000000,0.000000,1.000000,0.000000,21.000000,1.000000,0.000000,0.000000,0.000000
25%,57500.000000,0.000000,1.000000,0.000000,28.000000,3.000000,1.000000,1.000000,1.000000
50%,140000.000000,0.000000,2.000000,0.000000,33.000000,5.000000,2.000000,2.000000,3.000000
75%,240000.000000,1.000000,2.000000,1.000000,41.000000,6.000000,4.000000,4.000000,5.000000
max,630000.000000,1.000000,6.000000,2.000000,75.000000,10.000000,5.000000,15.000000,10.000000


Tambien encontramos que **Education** es mejor manejar los tipos de *5=unknown* y *6=unknown*. como un solo digito que es el 4=others

In [236]:
#reemplazamos 5 y 6 => 4
df_clients_A.Education = df_clients_A.Education.replace({5:4,6:4})

#comprobamos
df_clients_A.Education.value_counts()

2    287
1    260
3     98
4      7
Name: Education, dtype: int64

In [237]:
# verificamos los rangos
df_clients_A.describe()

,Limit_bal,Sex,Education,Marriage,Age,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
count,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000,652.000000
mean,170552.147239,0.423313,1.773006,0.450920,35.085890,4.736196,2.421779,2.590491,3.541411
std,129739.199740,0.494463,0.735750,0.547904,9.359618,2.161180,1.629703,2.946961,2.859721
min,10000.000000,0.000000,1.000000,0.000000,21.000000,1.000000,0.000000,0.000000,0.000000
25%,57500.000000,0.000000,1.000000,0.000000,28.000000,3.000000,1.000000,1.000000,1.000000
50%,140000.000000,0.000000,2.000000,0.000000,33.000000,5.000000,2.000000,2.000000,3.000000
75%,240000.000000,1.000000,2.000000,1.000000,41.000000,6.000000,4.000000,4.000000,5.000000
max,630000.000000,1.000000,4.000000,2.000000,75.000000,10.000000,5.000000,15.000000,10.000000


## 3. Clasificadores.